In [60]:
from keras import layers
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras import backend as K
#from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display

In [61]:
def res_block(y, nb_channels, _strides = (1,1), _project_shortcut=False):
    shortcut = y

    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    #y = layers.BatchNormalization()(y)
    y = layers.ReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    #y = layers.BatchNormalization()()

    if _project_shortcut or _strides != (1, 1):
        shortcut = layers.Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    y = layers.add([shortcut, y])
    #y = layers.LeakyReLU()(y)

    return y

In [62]:
def res_net(x, nb_channels, _strides=(1, 1)):
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    shortcut = x
    for _ in range(16):
        x = res_block(x, 64)

    x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='sigmoid')(x)
    x = layers.add([shortcut, x])
    
    return x

In [63]:
def conv_net(x, nb_channels, _strides=(1, 1)):
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    #x = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(x)
    
    return x

In [64]:
def post_net(y, nb_channels, _strides=(1, 1)):
    #y = layers.Conv2D(64, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(y)
    #y = layers.Conv2D(32, kernel_size=(3, 3), strides=_strides, padding='same', activation='relu')(y)
    y = layers.Conv2D(3, kernel_size=(3, 3), strides=_strides, padding='same', activation='tanh')(y)
    
    return y

In [65]:
import cv2
import numpy as np

def load_imgs(path, number, train_type):
    result=np.empty((number, 64, 64, 3), dtype="float64")
    for i in range(number):
        I = cv2.imread(path + "{:04}_{}.jpeg".format(i+1, train_type))
        result[i, :, :, :] = I
    return result/result.max()

In [66]:
#inport training data
dataNum = 1000
x1_train = load_imgs("./blurImg/", dataNum, 1)
x2_train = load_imgs("./blurImg/", dataNum, 2)
y_train = load_imgs("./blurImg/", dataNum, 0)

def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
def loss_wrapper(in_tensor1, in_tensor2):
    def gaussian_blur(in_tensor):
        # use large kernel to blur pred and in_tensor//
        return
        
    def custom_loss(y_true, y_pred):
        # or better implementation like fourier transformation
        return K.binary_crossentropy(y_true, y_pred) + K.reduce_mean(K.square(gaussian_blur(y_pred)-gaussian_blur(in_tensor1)))
    return custom_loss

In [67]:
img_a = layers.Input(shape=(64, 64, 3))
img_b = layers.Input(shape=(64, 64, 3))
#feature_a = conv_net(img_a, 3)
#feature_b = conv_net(img_b, 3)
feature_a = res_net(img_a, 3)
feature_b = res_net(img_b, 3)
merge = layers.concatenate([feature_a, feature_b])
aif = post_net(merge, 128)
gen = Model(inputs = [img_a, img_b], output = [aif])
gen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#gen.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
gen.summary()
#plot_model(gen, to_file='generator.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 64, 64, 64)   1792        input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_411 (Conv2D)             (None, 64, 64, 64)   1792        input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_378

/home/qian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  if __name__ == '__main__':


In [68]:
image_fake = gen([img_a, img_b])
dis = Sequential()
dis.add(layers.Conv2D(64, kernel_size=(3, 3),strides=(2, 2), padding='same'))
dis.add(layers.LeakyReLU())
#dis.add(layers.Dropout(0.25))
dis.add(layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2),padding='same'))
dis.add(layers.LeakyReLU())
#dis.add(layers.Dropout(0.25))
dis.add(layers.Conv2D(256, kernel_size=(3, 3), strides=(2, 2),padding='same'))
dis.add(layers.LeakyReLU())
#dis.add(layers.Dropout(0.25))
#dis.add(layers.Conv2D(1, kernel_size=(3, 3), padding='same'))

dis.add(layers.Flatten())
dis.add(layers.Dense(256))
dis.add(layers.Dense(2))
dis.add(layers.Activation('softmax'))
pred_prob = dis(image_fake)
dis.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dis.summary()
#plot_model(dis, to_file='discriminator.png')
make_trainable(dis, False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_446 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_447 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_448 (Conv2D)          (None, 8, 8, 256)         295168    
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 8, 8, 256)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 16384)             0         
__________

In [69]:
#pre-train discriminate network
make_trainable(dis,True)
Batch_size = 64
nb_epoch = 100
for epoch in range(nb_epoch):
    rand_idx = np.random.randint(0, x1_train.shape[0], size = Batch_size)
    img_batch1 = x1_train[rand_idx, :, :, :]
    img_batch2 = x2_train[rand_idx, :, :, :]
    y_batch = y_train[np.random.randint(0, y_train.shape[0], size = Batch_size), :, :, :]
    X = np.concatenate((y_batch, img_batch1))
    y = np.zeros([2*Batch_size,2])
    y[0:Batch_size, 0] = 1
    y[Batch_size:, 1] = 1

    dis.fit(X, y)
    #losses["d"].append(d_loss)

Epoch 1/1


InternalError: GPU sync failed

In [24]:
am = Model(inputs = [img_a, img_b], output = [pred_prob])
am.summary()
am.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#plot_model(am, to_file='adversary.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
model_3 (Model)                 (None, 64, 64, 3)    2444291     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 2)            2471811     model_3[1][0]                    
Total para

/home/qian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [12]:
def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(10,8))
        plt.plot(losses["d"], label='discriminitive loss')
        plt.plot(losses["g"], label='generative loss')
        plt.legend()
        plt.show()

In [13]:
# Train discriminator on generated images
losses = {"d":[], "g":[]}
Batch_size = 64
nb_epoch = 100
for epoch in range(nb_epoch):
    rand_idx = np.random.randint(0, x1_train.shape[0], size = Batch_size)
    img_batch1 = x1_train[rand_idx, :, :, :]
    img_batch2 = x2_train[rand_idx, :, :, :]
    y_batch = y_train[np.random.randint(0, y_train.shape[0], size = Batch_size), :, :, :]
    #gen.fit([x1_train, x2_train], y_train)
    gen_img = gen.predict([img_batch1, img_batch2])
    X = np.concatenate((y_batch, gen_img))
    y = np.zeros([2*Batch_size,])
    y[0:Batch_size] = 1
    y[Batch_size:] = 0
    make_trainable(dis,True)
    dis.fit(X, y)
    #losses["d"].append(d_loss)
    
    y2 = np.ones([Batch_size, ])
    # train Generator-Discriminator stack on input noise to non-generated output class
    make_trainable(dis,False)
    am.fit([img_batch1, img_batch1], y2) #same batch or ???
    #losses["g"].append(g_loss)
    #if epoch % 25 == 25 - 1:
    #    plot_loss(losses)

Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6948 - acc: 0.6250
Epoch 1/1
64/64 [==============================] - 5s 80ms/step - loss: 0.3957 - acc: 1.0000
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6768 - acc: 0.5703
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 0.7078 - acc: 0.4375
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.3744 - acc: 0.7656
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 3.2414 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.0239 - acc: 1.0000
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 6.6027 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 2.8408 - acc: 0.7109
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 1.1341e-04 - acc: 1.0000
Epoch 1/1
128/128 [==============================] - 

64/64 [==============================] - 2s 38ms/step - loss: 0.7361 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6957 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 37ms/step - loss: 0.7119 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6951 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 0.7270 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6938 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 0.6975 - acc: 0.0469
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6927 - acc: 0.4844
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 0.6542 - acc: 1.0000
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6940 - acc: 0.5078
Epoch 1/1
64/64 [==============================] - 2s 38ms/step

64/64 [==============================] - 2s 38ms/step - loss: 0.7532 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6965 - acc: 0.4688
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 0.6430 - acc: 0.9844
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6963 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 37ms/step - loss: 0.6777 - acc: 1.0000
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6980 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 3s 41ms/step - loss: 0.6628 - acc: 1.0000
Epoch 1/1
128/128 [==============================] - 1s 6ms/step - loss: 0.6943 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 39ms/step - loss: 0.6937 - acc: 0.0000e+00
Epoch 1/1
128/128 [==============================] - 1s 5ms/step - loss: 0.6944 - acc: 0.5000
Epoch 1/1
64/64 [==============================] - 2s 38ms/step - l

In [14]:
Batch_size = 64
nb_epoch = 100
for epoch in range(nb_epoch):
    rand_idx = np.random.randint(0, x1_train.shape[0], size = Batch_size)
    img_batch1 = x1_train[rand_idx, :, :, :]
    img_batch2 = x2_train[rand_idx, :, :, :]
    y_batch = y_train[rand_idx, :, :, :]
    gen.fit([img_batch1, img_batch2], y_batch)
    

#gen_img[0]
#gen_img.min()
#gen_img = gen.predict([x1_train, x2_train])    
#cv2.imwrite("a.jpg", 255.0*(gen_img[4]-gen_img[4].min())/(gen_img[4].max()-gen_img[4].min()))

Epoch 1/1
64/64 [==============================] - 5s 76ms/step - loss: 8.7845 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.3324 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 9.2125 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 32ms/step - loss: 8.9807 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 9.3150 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.9990 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 9.1182 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.3191 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 34ms/step - loss: 8.5524 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 34ms/step - loss: 9.0752 - acc: 0.0000e+00
Epoch 1/1
64/64 [=============

64/64 [==============================] - 2s 33ms/step - loss: 8.9552 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.9202 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.4503 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 35ms/step - loss: 8.4251 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 9.2337 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.7880 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.4816 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.4385 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.7507 - acc: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 2s 33ms/step - loss: 8.8783 - acc: 0.0000e+00
Epoch 1/1
64/64 [=======================